In [2]:
import pickle
output_name = "./nyt/label_embedding.pkl"
outp = open(output_name, "rb")
outp_dict = pickle.load(outp)

In [7]:
classes = ["politics","arts","business","science","sports"] # labels: [0, 1, 2, 3, 4]
target = [tup for tup in outp_dict if tup[0] in classes]

In [18]:
import torch
l = [tup[1] for tup in target]

In [31]:
l = torch.stack(l)

In [49]:
l = l.view(5, 1, 768)
l.shape

torch.Size([5, 1, 768])

In [48]:
query = target[1][1]
query = query.view(1, 1, 768)
query.shape

torch.Size([1, 1, 768])

In [38]:
from torch.nn import MultiheadAttention

In [40]:
multihead_attn = MultiheadAttention(768, 5)

In [50]:
attn_output, attn_output_weights = multihead_attn(query, l, l)

In [55]:
attn_output

tensor([[[-3.8710e-01, -7.4885e-02, -5.6313e-02, -2.4503e-01, -3.8560e-01,
          -2.4019e-01,  7.4876e-01, -2.6617e-01,  4.8928e-01, -4.1221e-01,
          -2.0205e-01,  1.0835e-01,  1.2628e-01, -3.9004e-01,  8.2782e-02,
          -2.5144e-02, -1.9333e-01, -4.0007e-03, -1.1078e-02,  3.8330e-01,
          -4.9787e-01,  1.8248e-02, -2.5645e-01, -2.0250e-01,  3.0712e-01,
           4.2614e-01, -2.6576e-01,  5.9220e-02,  3.6572e-02,  1.2182e-01,
          -8.9522e-02, -2.6313e-01, -2.8336e-01, -8.8508e-02, -1.1040e-02,
          -9.3293e-02,  3.0356e-01, -6.3142e-02,  1.6758e-02,  5.6823e-01,
           1.1161e-01,  3.2588e-01,  1.1857e-01, -2.7972e-01,  5.7268e-03,
           4.3544e-01,  5.3111e-02, -4.5232e-02,  1.5432e-01, -4.4534e-01,
          -5.6381e-01,  2.4137e-02,  3.2812e-01, -2.3415e-01,  1.5347e-01,
          -3.1453e-01,  4.1444e-01, -1.6766e-01, -1.7978e-01, -2.4453e-01,
           1.6053e-01, -1.3350e-01, -2.6992e-02, -1.0392e+00,  1.1396e-01,
          -3.2123e-02, -3

In [54]:
attn_output_weights

tensor([[[0.2167, 0.1950, 0.1876, 0.2059, 0.1949]]], grad_fn=<DivBackward0>)

In [58]:
from bert_modelling import CustomBERTClassifier

In [60]:
model = CustomBERTClassifier.from_pretrained(
    "bert-base-uncased",
    num_hidden_layers=5,
    num_labels=5,  # The number of output labels
    output_attentions=False,  # Whether the model returns attentions weights.
    output_hidden_states=False,  # Whether the model returns all hidden-states.
    class_embedding = l
)

TypeError: __init__() got an unexpected keyword argument 'class_embedding'

In [1]:
import torch
param_groups = torch.load("./results/nyt/keywords/level_0/pretrained_bert_ROOT.pt")
from bert_modelling import get_bert_based, load_bert_parameters
model = get_bert_based()
model = load_bert_parameters(model, param_groups)

print(model)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing CustomBERTClassifier: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing CustomBERTClassifier from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing CustomBERTClassifier from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CustomBERTClassifier were not initialized from the model checkpoint at bert-base-uncased and are newly ini

CustomBERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affi

In [2]:
from bert_modelling import Distillator
distillator = Distillator(model, temperature=2.0)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing CustomDistilBERTClassifier: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing CustomDistilBERTClassifier from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing CustomDistilBERTClassifier from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CustomDistilBERTClassifier were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias', 'mu

In [4]:
distillator.student

CustomDistilBERTClassifier(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin